In [ ]:
import requests
from PIL import Image
import pytesseract
import argparse
import cv2 as cv
import os
import shutil
import imutils
import numpy as np
import re
import json
from matplotlib import pyplot as plt
import tensorflow as tf
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Adapted from https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/label_image/label_image.py
#
# Copyright 2017 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

def load_graph(model_file):
    graph = tf.Graph()
    graph_def = tf.GraphDef()

    with open(model_file, "rb") as f:
        graph_def.ParseFromString(f.read())
    with graph.as_default():
        tf.import_graph_def(graph_def)

    return graph

def read_tensor_from_image_file(file_name,
                                input_height=299,
                                input_width=299,
                                input_mean=0,
                                input_std=255):
    input_name = "file_reader"
    output_name = "normalized"
    file_reader = tf.read_file(file_name, input_name)
    if file_name.endswith(".png"):
        image_reader = tf.image.decode_png(
            file_reader, channels=3, name="png_reader")
    elif file_name.endswith(".gif"):
        image_reader = tf.squeeze(
            tf.image.decode_gif(file_reader, name="gif_reader"))
    elif file_name.endswith(".bmp"):
        image_reader = tf.image.decode_bmp(file_reader, name="bmp_reader")
    else:
        image_reader = tf.image.decode_jpeg(
            file_reader, channels=3, name="jpeg_reader")
    float_caster = tf.cast(image_reader, tf.float32)
    dims_expander = tf.expand_dims(float_caster, 0)
    resized = tf.image.resize_bilinear(dims_expander, [input_height, input_width])
    normalized = tf.divide(tf.subtract(resized, [input_mean]), [input_std])
    sess = tf.Session()
    result = sess.run(normalized)

    return result

def load_labels(label_file):
    label = []
    proto_as_ascii_lines = tf.gfile.GFile(label_file).readlines()
    for l in proto_as_ascii_lines:
        label.append(l.rstrip())
    return label

def label_image(file_name):
    model_file = \
        "d2_classifier_graph.pb"
    label_file = "d2_classifier_labels.txt"
    input_height = 299
    input_width = 299
    input_mean = 0
    input_std = 255
    input_layer = "input"
    output_layer = "InceptionV3/Predictions/Reshape_1"
    
    graph = load_graph(model_file)
    t = read_tensor_from_image_file(
      'forOCR/' + file_name,
      input_height=input_height,
      input_width=input_width,
      input_mean=input_mean,
      input_std=input_std)

    input_name = "import/Placeholder"
    output_name = "import/final_result"
    input_operation = graph.get_operation_by_name(input_name)
    output_operation = graph.get_operation_by_name(output_name)

    with tf.Session(graph=graph) as sess:
        results = sess.run(output_operation.outputs[0], {
            input_operation.outputs[0]: t
        })
        results = np.squeeze(results)

        top_k = results.argsort()[-5:][::-1]
        labels = load_labels(label_file)
        for i in top_k:
            if results[i] > .9 and ( \
#                                     labels[i] == 'orbit' \
#                                  or \
#                                     labels[i] == 'pgcr' \
#                                  or \
                                    labels[i] == 'player menu' \
                                 or \
                                    labels[i] == 'roster' \
                                   ):
                
                if os.path.isdir('matches/' + labels[i]) == False:
                    os.mkdir('matches/' + labels[i])
                try:
                    shutil.copy('forOCR/' + file_name, 'matches/' + labels[i] + '/' + file_name)
                except:
                    print('error copying ocr file')
        try:
            os.remove('forOCR/' + file_name)
        except:
            print('error removing files')

def processQueue():
    for stream in queue:
        try:
            file_name = str(stream['channel']['_id']) + '.jpg'
            url = stream['preview']['template']
            url = url.replace('{width}', '1920')
            url = url.replace('{height}', '1080')
            r = requests.get(url)
            with open('forOCR/' + file_name, 'wb') as fd:
                for chunk in r.iter_content(chunk_size=128):
                    fd.write(chunk)
        except:
            print('issue retrieving 1080p screen')
            
def getStreams(offset = 0):
    twitchBaseUrl = 'https://api.twitch.tv/kraken/'
    twitchClientId = 'client_id=o8cuwhl23x5ways7456xhitdm0f4th0'
    
    if os.path.isdir('forOCR'):
        shutil.rmtree('forOCR')
    os.mkdir('forOCR')
    
    streamsUrl = twitchBaseUrl + 'streams?' + twitchClientId + '&game=Destiny%202&limit=100&offset=' + str(offset)
    r = requests.get(streamsUrl)
    json = r.json()
    if json['streams']:
        for stream in json['streams']:
            if ('recov' not in stream['channel']['status']):
                queue.append(stream)
    if json['_total'] > offset + 100:
        getStreams(offset + 100)
    else:
        processQueue()
        
def labelImages():
    for filename in os.listdir('forOCR'):
        label_image(filename)

def ocrPlayerScreen(filename):
    filepath = 'matches/player menu/' + filename
    im_rgb = cv.imread(filepath)
    im_crop1 = im_rgb[13:180,170:310]

    im = cv.cvtColor(im_crop1, cv.COLOR_BGR2GRAY)
    im = cv.threshold(im,150,255,cv.THRESH_BINARY_INV)[1]
    
    lower = np.array([0])
    upper = np.array([15])
    shapeMask = cv.inRange(im, lower, upper)

    cnts = cv.findContours(shapeMask.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts.reverse()

    for c in cnts:
        peri = cv.arcLength(c, True)
        approx = cv.approxPolyDP(c, 0.04 * peri, True)

        if len(approx) == 4:
            (x, y, w, h) = cv.boundingRect(approx)
            ar = w / float(h)
            
            if w > 7 and w < 13 and h <= 3 and ar >= 3.5:
                
                im_crop2 = im_rgb[y + 16:y + 60,x + 168:x+650]
                im_crop2 = cv.cvtColor(im_crop2, cv.COLOR_BGR2GRAY)
                im_crop2 = cv.threshold(im_crop2,150,255,cv.THRESH_BINARY_INV)[1]

                try:
                    text = pytesseract.image_to_string(im_crop2)
                    first = text.split('\n')[0]
                    ascii = first.encode('ascii', 'ignore').strip().split(' ')
                    sans_special = ''
                    for string in ascii:
                        if bool(re.match('[a-zA-Z0-9]+$', string)):
                            sans_special += string + ' '
                    stripped = sans_special.strip()
                    twitch_id = filename.split('.')[0]
                    if stripped:
                        if os.path.isdir('d2-stream-name-parser/' + stripped) == False:
                            os.mkdir('d2-stream-name-parser/' + stripped)
                        data = {}
                        try:
                            with open('d2-stream-name-parser/' + stripped + '/twitch.json') as f:
                                for i in f:
                                    data = json.loads(i)

                        except:
                            print('new file!')
                        if twitch_id in data:
                            data[twitch_id] += 1
                        else:
                            data[twitch_id] = 1
                        with open('d2-stream-name-parser/' + stripped + '/twitch.json', 'w+') as f:
                            f.write(json.dumps(data))
                except:
                    print('issue parsing text')
                break
    try:
        os.remove(filepath)
    except:
        print('error removing files')      

def ocrRoster(filename):
    filepath = 'matches/roster/' + filename
    im_rgb = cv.imread(filepath)
    im_gray = cv.cvtColor(im_rgb, cv.COLOR_BGR2GRAY)
    im_thresh = cv.threshold(im_gray,155,255,cv.THRESH_BINARY_INV)[1]

    lower = np.array([0])
    upper = np.array([15])
    shapeMask = cv.inRange(im_thresh, lower, upper)

    cnts = cv.findContours(shapeMask.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)

    for c in cnts:
        peri = cv.arcLength(c, True)
        approx = cv.approxPolyDP(c, 0.04 * peri, True)

        if len(approx) == 4:
            (x, y, w, h) = cv.boundingRect(approx)
            ar = w / float(h)

            if ar >= .1 and ar <= .3 and h > 25:
                crop = im_thresh[approx[2][0][1] - 40 : approx[2][0][1] , approx[0][0][0] - 415 : approx[0][0][0] - 10]
                try:
                    text = pytesseract.image_to_string(crop)
                    first = text.split('\n')[0]
                    ascii = first.encode('ascii', 'ignore').strip().split(' ')
                    sans_special = ''
                    for string in ascii:
                        if bool(re.match('[a-zA-Z0-9]+$', string)):
                            sans_special += string + ' '
                    stripped = sans_special.strip()
                    q_check = stripped.split(' ')
                    # Sometimes, the audio icon reads as a 'q', so...
                    if q_check[-1] == 'q':
                        q_check[-1] = ''
                        stripped = ' '.join(q_check).strip()
                    twitch_id = filename.split('.')[0]
                    if stripped:
                        if os.path.isdir('d2-stream-name-parser/' + stripped) == False:
                            os.mkdir('d2-stream-name-parser/' + stripped)
                        data = {}
                        try:
                            with open('d2-stream-name-parser/' + stripped + '/twitch.json') as f:
                                for i in f:
                                    data = json.loads(i)

                        except:
                            print('new file!')
                        if twitch_id in data:
                            data[twitch_id] += 1
                        else:
                            data[twitch_id] = 1
                        with open('d2-stream-name-parser/' + stripped + '/twitch.json', 'w+') as f:
                            f.write(json.dumps(data))
                except:
                    print('issue parsing text')
                break
    try:
        os.remove(filepath)
    except:
        print('error removing files')

        
queue = []

getStreams()
labelImages()

for filename in os.listdir('matches/player menu'):
    if not filename.startswith('.'):
        ocrPlayerScreen(filename)
        
for filename in os.listdir('matches/roster'):
    if not filename.startswith('.'):
        ocrRoster(filename)

import sched, time

s = sched.scheduler(time.time, time.sleep)
def do_something(sc): 
    queue = []
    getStreams()
    labelImages()

    for filename in os.listdir('matches/player menu'):
        if not filename.startswith('.'):
            ocrPlayerScreen(filename)

    for filename in os.listdir('matches/roster'):
        if not filename.startswith('.'):
            ocrRoster(filename)
            
    s.enter(0, 1, do_something, (sc,))

s.enter(0, 1, do_something, (s,))
s.run()